<a href="https://colab.research.google.com/github/julius-kanani/whisper-transcript-generator-mvp/blob/main/Whisper%20Transcript%20Generator%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# whisper_transcript_generator_mvp.ipynb

# 📌 Install Required Libraries
!pip install -q whisper python-docx yt-dlp pypandoc
!pip install -q git+https://github.com/openai/whisper.git
!sudo apt update -y && sudo apt install -y ffmpeg pandoc

# 📌 Import Libraries
import whisper
import yt_dlp
import os
import re
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# 📌 Create output directory
OUTPUT_DIR = "whisper_transcripts"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 📌 Dynamic Topics Dictionary (editable)
Topics = {
    1: "Introduction to Strategic Management",
    2: "Key Components of Strategic Management",
    3: "Strategic Thinking & Competitive Advantage",
    4: "Internal Business Environment",
    5: "External Business Environment",
    6: "Environmental Analysis Tools",
    7: "Mid-Semester Exam Preparation",
    8: "Strategic Choices",
    9: "Managing Strategic Change",
    10: "Formulation & Implementation",
    11: "Evaluation & Control",
    12: "Global Strategies",
}

# 📌 UI Widgets
week_no = widgets.IntSlider(description='Week:', min=1, max=12, value=1)
course_name = widgets.Text(description='Course Name:', value='Strategic Management')
course_code = widgets.Text(description='Course Code:', value='MBA 6080A')
platform = widgets.Dropdown(options=['YouTube', 'Panopto', 'Local Upload'], description='Platform:')
youtube_url = widgets.Text(description='Video URL:', placeholder='Paste YouTube/Panopto URL')
audio_upload = widgets.FileUpload(accept='.mp3', description='Upload Audio')
process_button = widgets.Button(description='Generate Transcript', button_style='success')
output = widgets.Output()

# 📌 Display UI
display(week_no, course_name, course_code, platform, youtube_url, audio_upload, process_button, output)

# 📌 Metadata generator
def generate_metadata():
    return {
        "WeekNo": week_no.value,
        "CourseName": course_name.value.upper(),
        "CourseCode": course_code.value.upper(),
        "Platform": platform.value.upper(),
        "WeekTopic": Topics.get(week_no.value, "Undefined Topic"),
        "TranscriptType": "Overview Video".upper(),
        "TranscriptAuthor": "Dr. Juliana Namada",
    }

# 📌 YouTube/Panopto downloader
def download_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(OUTPUT_DIR, 'downloaded_audio.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }]
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return os.path.join(OUTPUT_DIR, "downloaded_audio.mp3")

# 📌 Split transcript into readable paragraphs
def split_text_into_paragraphs(text, max_sentences=5):
    sentences = re.split(r'(?<=[.!?]) +', text)
    paragraphs, current = [], []
    for sentence in sentences:
        current.append(sentence)
        if len(current) >= max_sentences:
            paragraphs.append(' '.join(current))
            current = []
    if current:
        paragraphs.append(' '.join(current))
    return paragraphs

# 📌 DOCX generator
def generate_docx(text, meta):
    doc = Document()
    doc.core_properties.title = f"{meta['Platform']} Video Transcript"
    doc.core_properties.author = meta['TranscriptAuthor']
    doc.core_properties.keywords = f"{meta['CourseCode']}, {meta['WeekNo']}, {meta['Platform']}, {meta['CourseName']}, {meta['WeekTopic']}"

    headings = [
        f"{meta['Platform']} VIDEO TRANSCRIPT",
        f"{meta['CourseCode']} – {meta['CourseName']}",
        f"WEEK {meta['WeekNo']} {meta['TranscriptType']} Transcript",
        f"{meta['WeekTopic']}"
    ]

    for heading in headings:
        p = doc.add_paragraph()
        run = p.add_run(heading)
        p.style = 'Heading 1'
        run.font.size = Pt(18)
        run.bold = True
        p.alignment = WD_ALIGN_PARAGRAPH.CENTER

    paragraphs = split_text_into_paragraphs(text)
    for paragraph in paragraphs:
        p = doc.add_paragraph(paragraph)
        p.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY
        for run in p.runs:
            run.font.size = Pt(16)

    filename = f"W{meta['WeekNo']}_{meta['TranscriptType']} - {meta['CourseCode']}_{meta['CourseName']}_{meta['Platform']}_Video_Transcript.docx"
    file_path = os.path.join(OUTPUT_DIR, filename)
    doc.save(file_path)
    return file_path

# 📌 Main pipeline function
def process_pipeline(b):
    clear_output()
    display(week_no, course_name, course_code, platform, youtube_url, audio_upload, process_button, output)
    with output:
        print("🔄 Starting transcript generation pipeline...")

        meta = generate_metadata()
        model = whisper.load_model("medium")

        # Handle audio input
        if platform.value in ["YouTube", "Panopto"] and youtube_url.value:
            print(f"🎥 Downloading {platform.value} audio...")
            audio_file = download_audio(youtube_url.value)
        elif platform.value == "Local Upload" and audio_upload.value:
            print("📥 Processing uploaded audio...")
            fname = list(audio_upload.value.keys())[0]
            audio_file = os.path.join(OUTPUT_DIR, fname)
            with open(audio_file, 'wb') as f:
                f.write(audio_upload.value[fname]['content'])
        else:
            print("❌ Please provide a valid video URL or upload an audio file.")
            return

        print("🎙️ Transcribing audio with Whisper...")
        result = model.transcribe(audio_file, language='en')
        transcription_text = result['text']

        print("📝 Generating Word document...")
        docx_file = generate_docx(transcription_text, meta)

        print("✅ Transcript generation complete! Download your file below 👇")
        files.download(docx_file)

process_button.on_click(process_pipeline)